# 0) LIBRARIES & DATASET

In [ ]:
!pip install ydata-profiling efficient-apriori recordlinkage
!pip install outlier-utils
!git clone https://github.com/SusannaPaoletti/DataQuality

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import recordlinkage
%matplotlib inline

In [ ]:
import sklearn
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import ensemble
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import ShuffleSplit, cross_val_score

from DataQuality import scripts_for_E5 as s

In [ ]:
# load dirty dataset
data = pd.read_csv("https://raw.githubusercontent.com/SusannaPaoletti/DataQuality/refs/heads/main/Comune-di-Milano-Attivita-commerciali-di-media-e-grande-distribuzione.csv", sep=';', encoding='utf-16')
data


# 1) DATA EXPLORATION & PROFILING

In [ ]:
#display dataset columns
data.columns

In [ ]:
#display the first 5 rows
data.head()

In [ ]:
# display the shape of the dataset
data.shape

In [ ]:
#description of the numerical variables
data.describe()
#note that min(Superficie totale) = 0 which is clearly an error

In [ ]:
# display column types
data.dtypes

In [ ]:
# number of distinct values of Tipo Via
data['Tipo Via'].nunique()

In [ ]:
# distinct values of Tipo Via
data['Tipo Via'].unique()

In [ ]:
# number of distinct values of Settore Merceologico
data['Settore Merceologico'].nunique()

In [ ]:
# distinct values of Settore Merceologico
data['Settore Merceologico'].unique()

In [ ]:
# count occurrences for each unique value of Settore Merceologico
settore_count = data['Settore Merceologico'].value_counts()
settore_count

In [ ]:
# count occurrences for each unique value of Insegna
insegna_count = data['Insegna'].value_counts()
insegna_count

In [ ]:
# distinct values of ZD
data['ZD'].unique()

DUPLICATES:

In [ ]:
# display exact duplicates
duplicates = data.duplicated()
print("There's", duplicates.sum(), "exact duplicate:")
data[data.duplicated()]

In [ ]:
# percentage of duplicate rows in the dataset
data.duplicated().sum()/data.shape[0]*100

NULL VALUES:

In [ ]:
# missing values in the dataset
data.isnull()

In [ ]:
# display null values per column
data.isnull().sum()

In [ ]:
# total number of null values
NULL = data.isnull().sum().sum()
NULL

NOT NULL VALUES:

In [ ]:
# display not null values per column
data.count()

In [ ]:
# total number of values in the dataset
TOT = data.shape[0]*data.shape[1]
TOT

In [ ]:
# select numerical columns from the dataset
NUM = list(data.select_dtypes(include=['int64','float64']).columns)
DATA_NUM = data[NUM]
DATA_NUM

In [ ]:
# check the correlation between numerical columns
corr = DATA_NUM.corr(method ='pearson')
sns.heatmap(corr)
plt.show()
# note: as we expected there's significant correlation between Superficie vendita, S. totale e S. altri usi but also between ZD and Codice Via

In [ ]:
# plot value distributions per each column
DATA = data.select_dtypes(include=['int64','float64'])
for col in DATA.columns:
    print("Histogram for "+col+":")
    plt.figure(figsize=(20,6))

    DATA[col].hist()
    print("\n\n")
    plt.show()

In [ ]:
# generate a profile report
PROFILE = ProfileReport(data, title="Pandas Profiling Report")
PROFILE

# 2) DATA QUALITY ASSESSMENT

COMPLETENESS EVALUATION

In [ ]:
# calculate data completeness ratio
COMPLETENESS = (TOT - NULL) / TOT
COMPLETENESS

In [ ]:
# format completeness as a percentage with one decimal place
COMPLETENESS = '{0:.1f}%'.format(COMPLETENESS*100)
print(COMPLETENESS)

ACCURACY EVALUATION:

In [ ]:
# define the range of incorrect values based on dataset documentation
RANGE_INCORRECT = range(0, 250)

# count entries in 'Superficie totale' that fall within the incorrect range
INCORRECT_Superficie_Totale = sum(1 for item in data['Superficie totale'] if item in RANGE_INCORRECT)

INCORRECT_Superficie_Totale

In [ ]:
# count the number of non-null entries in 'Superficie totale'
NOT_NULL_Superficie_totale= data['Superficie totale'].count()
NOT_NULL_Superficie_totale

In [ ]:
# calculate the accuracy for 'Superficie totale' based on incorrect and non-null values
ACCURACY_SUPERFICIE_TOT = (data.shape[0] - INCORRECT_Superficie_Totale) / NOT_NULL_Superficie_totale

# format accuracy as a percentage with one decimal place
ACCURACY_SUPERFICIE_TOT = '{0:.1f}%'.format(ACCURACY_SUPERFICIE_TOT * 100)

print(ACCURACY_SUPERFICIE_TOT)


CONSISTENCY EVALUATION:

In [ ]:
# integrity rule: superficie vendita + superficie altri usi = superficie totale
data['consistency'] = np.where(data['Superficie vendita'] + data['Superficie altri usi'].fillna(0) != data['Superficie totale'], 0, 1)

In [ ]:
# sum the values in the 'consistency' column
CONSISTENT = data['consistency'].sum()
CONSISTENT

In [ ]:
# count consistent rows
COUNT = data['consistency'].count()
COUNT

In [ ]:
# calculate consistency as the ratio of consistent values to total count
CONSISTENCY = CONSISTENT / COUNT

# format consistency as a percentage with one decimal place
CONSISTENCY = '{0:.1f}%'.format(CONSISTENCY * 100)

print(CONSISTENCY)


In [ ]:
# drop the 'consistency' column from the dataset
data = data.drop(columns='consistency')

TIMELINESS EVALUATION won't be considered because we don't have the temporal context of data

# 3) DATA CLEANING (DATA TRANSFORMATION)

In [ ]:
# convert 'Superficie altri usi' to type int
data['Superficie altri usi'] = data['Superficie altri usi'].fillna(0).astype(int)

In [ ]:
# drop exact duplicates
data = data.drop_duplicates()

In [ ]:
# extracts unique values of 'Tipo Via'
unique_values = data['Tipo Via'].dropna().unique()
print("Valori univoci nella colonna 'Tipo via':", unique_values)

In [ ]:
# extracts new 'Tipo via' from 'Ubicazione'
data[['Tipo via estratta', 'Primo split']] = data['Ubicazione'].str.split(" ", n=1, expand=True)
data

In [ ]:
# Valid 'Tipo Via' values
tipo_via = ['PZA', 'RIP', 'VIA', 'VLE', 'PLE', 'CSO', 'GLL', 'FOR', 'LGO', 'LARGO', 'BST', 'PAS', 'PTA', 'ALZ']

# Select rows with invalid 'Tipo Via' values
non_valide = data[~data['Tipo via estratta'].isin(tipo_via)]

# Remove invalid rows if any are found
if not non_valide.empty:
    print("Invalid tuples found:")
    print(non_valide[['Ubicazione', 'Tipo via estratta']])

    data = data.drop(non_valide.index)

In [ ]:
# define column names for comparison
colonna1 = 'Tipo Via'
colonna2 = 'Tipo via estratta'

# extract rows where values in the columns differ
righe_differenti = data[data[colonna1] != data[colonna2]]

# display rows with different values in the specified columns
righe_differenti[[colonna1, colonna2]]

In [ ]:
# drop old column 'Tipo Via'
data = data.drop('Tipo Via', axis=1)

In [ ]:
# extract new 'Via' values from 'Primo split' using a regular expression
data['Via estratta'] = data['Primo split'].str.extract(r'^([A-Z\s\.\']+?)(?=\s(?:N\.|\sN |\snum\.|\(|[a-z]))')

# remove extracted part from 'Primo split' and clean up the string
data['Primo split'] = data['Primo split'].str.replace(r'^([A-Z\s\.\']+?)(?=\s(?:N\.|\sN |\snum\.|\(|[a-z]))', '', regex=True).str.strip()

data

In [ ]:
# define column names for comparison
colonna1 = 'Via'
colonna2 = 'Via estratta'

# extract rows where values in the columns differ
righe_differenti = data[data[colonna1] != data[colonna2]]

# display rows with different values in the specified columns
righe_differenti[[colonna1, colonna2]]

In [ ]:
# drop old column 'Via'
data = data.drop('Via', axis=1)

In [ ]:
# extract new 'Civico' values from 'Primo split' using a regular expression
data['Civico estratto'] = data['Primo split'].str.extract(r'(\d[\/\w]*)(?=\s|;)', expand=True)

# remove leading zeros from the extracted 'Civico' values
data['Civico estratto'] = data['Civico estratto'].str.replace(r'^0{1,2}', '', regex=True)

data

In [ ]:
# convert 'Civico' column to integer, filling NaN values with -1
data['Civico'] = data['Civico'].fillna(-1).astype(int)

# convert both 'Civico' and 'Civico estratto' columns to string type
data['Civico'] = data['Civico'].astype(str)
data['Civico estratto'] = data['Civico estratto'].astype(str)

# extract rows where values in the columns differ
righe_differenti = data[data['Civico'] != data['Civico estratto']]

# display rows with different values in the specified columns
righe_differenti[['Civico', 'Civico estratto']]

In [ ]:
# drop old calumn 'Civico'
data = data.drop('Civico', axis=1)

In [ ]:
# extract new 'ZD' values from 'Primo split' using a regular expression
data['(z.d.) estratto'] = data['Primo split'].str.extract(r'\(z\.d\.\s*(\d+)\)', expand=True)
data

In [ ]:
# convert '(z.d.) estratto' column to integer type
data['(z.d.) estratto'] = data['(z.d.) estratto'].astype(int)

# extract rows where values in the columns differ
righe_differenti = data[data['ZD'] != data['(z.d.) estratto']]

# display rows with different values in the specified columns
righe_differenti[['ZD', '(z.d.) estratto']]

In [ ]:
# drop old columns
data = data.drop('ZD', axis=1)
data = data.drop('Ubicazione', axis=1)
data = data.drop('Primo split', axis=1)

In [ ]:
# rename new columns
data.rename(columns={'Via estratta': 'Via', 'Civico estratto': 'Civico', '(z.d.) estratto': 'ZD', 'Tipo via estratta': 'Tipo Via',}, inplace=True)
data

# 4) DATA CLEANING (ERROR DETECTION&CORRECTION MISSING VALUES)

In [ ]:
# Function to update 'Superficie altri usi' and fix inconsistencies with 'Superficie totale'
def aggiorna_superficie(row):
    if (row['Superficie altri usi']) == 0 and row['Superficie totale'] > row['Superficie vendita']:
        # If 'Superficie altri usi' is 0 and 'Superficie totale' is greater than 'Superficie vendita'
        row['Superficie altri usi'] = row['Superficie totale'] - row['Superficie vendita']  # Fill NaN values for 'Superficie altri usi'
    elif (row['Superficie altri usi']) == 0 and row['Superficie totale'] == row['Superficie vendita']:
        row['Superficie altri usi'] = 0  # Fill NaN values for 'Superficie altri usi'
    elif (row['Superficie altri usi']) == 0 and row['Superficie totale'] < row['Superficie vendita']:
        row['Superficie totale'] = row['Superficie vendita']  # Adjust 'Superficie totale' when it's less than 'Superficie vendita'
        row['Superficie altri usi'] = 0  # Fill NaN values for 'Superficie altri usi'
    elif (row['Superficie altri usi']) != 0:
        row['Superficie totale'] = row['Superficie vendita'] + row['Superficie altri usi']  # Correct 'Superficie totale' if 'Superficie altri usi' is not 0
    return row

In [ ]:
# apply the 'aggiorna_superficie' function to each row of the dataset
data = data.apply(aggiorna_superficie, axis=1)

In [ ]:
# print the updated data with changes to the 'Superficie totale' column
print("Dati aggiornati con le modifiche alla colonna 'Superficie totale':")
data[['Superficie vendita', 'Superficie altri usi', 'Superficie totale']]

ML

In [ ]:
# display the number of missing values in each column
data.isna().sum()

In [ ]:
# remove rows where both 'Settore Merceologico' and 'Insegna' are NaN
data = data.dropna(subset=['Settore Merceologico', 'Insegna'], how='all')

In [ ]:
# fill NaN values in the 'Insegna' column with 'Non Specificato'
data['Insegna'] = data['Insegna'].fillna('Non Specificato')
data

In [ ]:
# save the updated dataset to a CSV file without including the index
data.to_csv('dataset_aggiornato.csv', index=False)

In [ ]:
# print rows where 'Settore Merceologico' is NaN before imputation
print("Prima dell'imputazione:")
data[data['Settore Merceologico'].isna()]

Logistic Regression for categorical variables

In [ ]:
# rimozione delle colonne non rilevanti
df = data.drop(['Codice Via', 'Tipo Via', 'Via', 'Civico', 'ZD'], axis=1)

# separazione delle colonne categoriche e numeriche
CAT = list(df.select_dtypes(include=['object']).columns)
NUM = list(df.select_dtypes(include=['int64', 'float64']).columns)
CAT.remove('Settore Merceologico')  # escludiamo la variabile target

df

In [ ]:
# Create new columns with imputed values using standard techniques
for feature in df.columns:
    if feature == 'Settore Merceologico':
        continue

    # Initialize new column for imputed values
    df[feature + '_imp'] = df[feature]

    # For numerical features, impute with median
    if feature in NUM:
        df.loc[df[feature].isnull(), feature + '_imp'] = df[feature].median()

    # For categorical features, impute with mode
    elif feature in CAT:
        df.loc[df[feature].isnull(), feature + '_imp'] = df[feature].mode()[0]

In [ ]:
# Impute missing values using Machine Learning
IMP_DATA = df.copy()

for feature in ['Settore Merceologico']:  # Target to predict
    IMP_DATA['IMP_' + feature] = df[feature]
    parameters = list(set(df.columns) - {'Settore Merceologico', 'IMP_' + feature})

    # Encode categorical variables
    X = s.encoding_categorical_variables(df[parameters])

    # Filter valid and missing rows
    valid_rows = df[feature].notnull()
    missing_rows = df[feature].isnull()

    # Logistic regression model
    model = linear_model.LogisticRegression(max_iter=1000)
    model.fit(X.loc[valid_rows], df[feature][valid_rows])

    # Predict missing values
    predictions = model.predict(X.loc[missing_rows])

    # Replace missing values with predictions
    IMP_DATA.loc[missing_rows, 'IMP_' + feature] = predictions

# Filter and print rows where the 'Settore Merceologico' was imputed
modified_rows = IMP_DATA[IMP_DATA['Settore Merceologico'].isnull() & IMP_DATA['IMP_Settore Merceologico'].notnull()]

modified_rows

In [ ]:
# print the first 10 predicted values for the imputed 'Settore Merceologico'
print(predictions[:10])

Random Forest Classifier

In [ ]:
# Remove irrelevant columns from the dataset
df = data.drop(['Codice Via', 'Tipo Via', 'Via', 'Civico', 'ZD'], axis=1)

# Separate categorical and numerical columns
CAT = list(df.select_dtypes(include=['object']).columns)
NUM = list(df.select_dtypes(include=['int64', 'float64']).columns)

# Simple Imputation (using mode for categorical variables)
for feature in ['Settore Merceologico']: # Changed

    df[feature + '_imp'] = df[feature]

    # For numerical features, impute with median
    if feature in NUM:
        df.loc[df[feature].isnull(), feature + '_imp'] = df[feature].median()

    # For categorical features, impute with mode
    if feature in CAT:
        df.loc[df[feature].isnull(), feature + '_imp'] = df[feature].mode()[0]

# Imputation with Random Forest
IMP_DATA = df.copy()

for feature in ['Settore Merceologico']:  # Target to predict
    IMP_DATA['IMP_' + feature] = df[feature]
    parameters = list(set(df.columns) -  {feature + '_imp'} - {'Settore Merceologico', 'IMP_' + feature})

    # Encode categorical variables
    X = s.encoding_categorical_variables(df[parameters])

    # Filter valid and missing rows
    valid_rows = df[feature].notnull()
    missing_rows = df[feature].isnull()

    # Random Forest model for imputation
    if feature in NUM:
        model = ensemble.RandomForestRegressor()
    elif feature in CAT:
        model = ensemble.RandomForestClassifier()

    # Train the model on non-null data
    model.fit(X.loc[valid_rows], df[feature][valid_rows])

    # Predict missing values
    predictions = model.predict(X.loc[missing_rows])

    # Replace missing values with predictions
    IMP_DATA.loc[missing_rows, 'IMP_' + feature] = predictions

# Print the rows that were modified
modified_rows = IMP_DATA[IMP_DATA['Settore Merceologico'].isnull() & IMP_DATA['IMP_Settore Merceologico'].notnull()]
modified_rows

In [ ]:
# substitute the imputed data inside the main dataset
data['Settore Merceologico'] = IMP_DATA['IMP_Settore Merceologico']
data

In [ ]:
# Count the total number of null values in the dataset
NULL = data.isnull().sum().sum()
NULL

# 5) DATA CLEANING (ERROR DETECTION&CORRECTION OUTLIERS)

In [ ]:
# Identify and fix wrong values for Civico
# Note : we certainly know that the only invalid civico is '0' based on how we extracted it from 'Ubicazione'
def drop_invalid_civico(df):
    # Identify rows where Civico is '0'
    invalid_rows = df[df['Civico'].astype(str) == "0"]

    # Print invalid rows if any are found
    if not invalid_rows.empty:
        print("Rows where 'Civico' is '0':")
        print(invalid_rows)
    else:
        print("No rows found with 'Civico' equal to '0'.")

    # Drop rows with Civico = '0' and return the updated DataFrame
    return df[df['Civico'].astype(str) != "0"]

In [ ]:
# Drop rows with invalid Civico
data=drop_invalid_civico(data)

In [ ]:
# Check for negative values in surface columns
colonne_superfici = ['Superficie totale', 'Superficie vendita', 'Superficie altri usi']

for colonna in colonne_superfici:
    valori_negativi = data[data[colonna] < 0]
    if not valori_negativi.empty:
        print(f"Negative values found in column '{colonna}':")
        print(valori_negativi)
    else:
        print(f"No negative values in column '{colonna}'.")

In [ ]:
# Plot histogram for 'Superficie totale'
plt.figure(figsize=(10, 6))  # Set the figure size

# Seaborn histogram
sns.distplot(data["Superficie totale"], hist=True, kde=False, bins=int(180/5), color = 'blue', hist_kws={'edgecolor':'black'})

# Add labels and title
plt.title('Distribution of Superficie totale', fontsize=16)
plt.xlabel('Superficie totale', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Show grid for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show the plot
plt.show()


STATISTIC-BASED:

Z-Score e STD are not reliable since these parametric methods assume the values follow a Normal distribution.

In [ ]:
# Function to detect outliers using Z-score method
def ZS(data, threshold):
    mean = np.mean(data)
    sd = np.std(data)
    outliers = []
    for i in data:
        z = (i - mean) / sd
        if abs(z) > threshold:
            outliers.append(i)
    return outliers

In [ ]:
# Apply ZS function to detect outliers in 'Superficie totale' column with threshold of 3
ZS(data['Superficie totale'], 3)

Standard Deviation

In [ ]:
# Function to detect outliers using standard deviation method
def STD(data, th):
    mean = data.mean()
    std = data.std()
    V1 = mean + th * std
    V2 = mean - th * std
    outliers = []
    outliers_ind = []
    for d in data:
        if (d > V1) | (d < V2):
            outliers.append(d)

    return outliers

In [ ]:
# Apply STD function to detect outliers in 'Superficie totale' column with threshold of 3
STD(data['Superficie totale'], 3)

In [ ]:
# Function to detect outliers using Robust Z-score based on Median and Median Absolute Deviation (MAD)
def ZSB(data, threshold):
    # Robust Zscore as a function of median and median
    # median absolute deviation (MAD) defined as
    # z-score = |x – median(x)| / mad(x)
    median = np.median(data)
    print("Median: ", median)
    median_absolute_deviation = np.median(np.abs(data - median))
    modified_z_scores = (data - median) / median_absolute_deviation
    outliers = data[np.abs(modified_z_scores) > threshold]

    print("The detected outliers are: ", str(outliers))

In [ ]:
ZSB(data['Superficie totale'], 3)

In [ ]:
# Function to detect outliers using percentiles method (1st and 99th percentile)
def PERC(data):
    V1 = np.percentile(data, 99)
    V2 = np.percentile(data , 1)
    outliers = []
    for d in data:
        if (d > V1) | (d < V2):
            outliers.append(d)
    return outliers

In [ ]:
# Apply PERC function to detect outliers in 'Superficie totale' column
PERC(data['Superficie totale'])


IQR (non parametric approach)

In [ ]:
def IQR(data):
    sorted(data)
    Q1, Q3 = np.percentile(data, [25, 75])
    IQR = Q3 - Q1
    lower_range = Q1 - (2 * IQR)
    upper_range = Q3 + (4 * IQR) # increased parameter since the value distribution is skewed
    outliers = data[((data < lower_range) | (data > upper_range))]
    # print outliers
    print("The detected outliers are: ", str(outliers))

In [ ]:
IQR(data['Superficie totale'])

MODEL-BASED: KNN

In [ ]:
# Creating a new dataframe with only the surface data
data_superfici = data.loc[:, ['Superficie vendita', 'Superficie altri usi', 'Superficie totale']]
# show the structure of the new dataframe
print(data_superfici.head())


In [ ]:
#Training
X = data_superfici.values
X

In [ ]:
#Instantiate model (KNN)
KNN = NearestNeighbors(n_neighbors = 3) #finds the 3 nearest neighbors for each sample

In [ ]:
#Fit model
KNN.fit(X)

In [ ]:
#Distances and indexes of k-neaighbors from model outputs
distances, indexes = KNN.kneighbors(X)
distances.mean(axis = 1)

In [ ]:
#Plot mean of k-distances of each observation
plt.plot(distances.mean(axis =1))

In [ ]:
global_mean_dist=np.mean(distances.mean(axis =1))
print(global_mean_dist)

In [ ]:
# Exclude points at a high distance
# threshold: cutoff values > 95th percentile of the mean distance (since value distribution is skewed)
OUTLIER_INDEX = np.where(distances.mean(axis = 1) > np.percentile(distances.mean(axis =1), 95))
OUTLIER_INDEX

In [ ]:
#Filter outlier values
OUTLIERS_VALUES = data_superfici.iloc[OUTLIER_INDEX]
OUTLIERS_VALUES

In [ ]:
#Plot outliers
plt.scatter(data_superfici['Superficie totale'], data_superfici['Superficie vendita'], color = "royalblue")
plt.scatter(OUTLIERS_VALUES['Superficie totale'], OUTLIERS_VALUES['Superficie vendita'], color = "hotpink")
plt.show()

# 6) DATA CLEANING (DATA DEDUPLICATION)

**1. drop exact duplicates**

In [ ]:
# show exact duplicates
data[data.duplicated()]

In [ ]:
# drop duplicates
data = data.drop_duplicates()

**2. search for candidate duplicates with Record Linkage**

method 1: indexing with Blocking

In [ ]:
#create a copy of the dataset
data1=data.copy()

In [ ]:
#load the recordlinkage.Index class
indexer = recordlinkage.Index()

In [ ]:
# definition of the blocking criteria by choosing the attributes who discriminate more
indexer.block(['Tipo Via', 'Via', 'Civico'])
candidate_links=indexer.index(data1)

In [ ]:
# show the comparisons found
print(len(candidate_links))
candidate_links

In [ ]:
# definition of the rules to compare the candidate pairs
compare_cl = recordlinkage.Compare()
compare_cl.exact('ZD', 'ZD', label='ZD')
compare_cl.exact('Codice Via', 'Codice Via', label='Codice Via')
compare_cl.string('Insegna', 'Insegna', method='jarowinkler', threshold=0.80, label='Insegna')
compare_cl.numeric('Superficie vendita', 'Superficie vendita', method='linear', offset=15, label='Superficie vendita')
compare_cl.numeric('Superficie altri usi', 'Superficie altri usi', method='linear', offset=15, label='Superficie altri usi')
compare_cl.numeric('Superficie totale', 'Superficie totale', method='linear', offset=30, label='Superficie totale')
features = compare_cl.compute(candidate_links, data1)
features

In [ ]:
# Count and sort the number of record pairs by their total similarity score
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
# Compare the matches
matches = features[features.sum(axis=1) >4]

print(len(matches))

matches

qui dobbiamo decidere quali righe droppare

In [ ]:
# display candidate duplicates
data.loc[[12, 20]]

In [ ]:
# Drop line 12
# We retained duplicate 20 because the 'Settore Merceologico' value is more general
data = data.drop([12])

In [ ]:
# display candidate duplicates
data.loc[[363, 364]]

In [ ]:
# Calculate the mean of "Superficie vendita" and "Superficie totale" for indices 363 and 364
mean_superficie_vendita = data.loc[[363, 364], "Superficie vendita"].mean()
mean_superficie_totale = data.loc[[363, 364], "Superficie totale"].mean()

# Update the row 363 with the mean values
data.loc[363, "Superficie vendita"] = math.ceil(mean_superficie_vendita)
data.loc[363, "Superficie totale"] = math.ceil(mean_superficie_totale)

# Drop the duplicate row (index 364)
data = data.drop(index=364)

In [ ]:
data.loc[[363]]

In [ ]:
# display candidate duplicates
data.loc[[299, 727]]

In [ ]:
# Drop line 299
# We retained duplicate 727 because is more specific on the 'Insegna' value
data = data.drop([299])

In [ ]:
data.loc[[821, 612]]

In [ ]:
# Drop line 612
# We retained duplicate 821 because it aligns with the most recent 'Insegna' name displayed on the external source Google Maps (accessed on January 10, 2025)
data = data.drop([612])

In [ ]:
# display candidate duplicates
data.loc[[875, 874]]

In [ ]:
# Calculate the mean of "Superficie vendita" and "Superficie totale" for indices 875 and 874
mean_superficie_vendita = data.loc[[875, 874], "Superficie vendita"].mean()
mean_superficie_totale = data.loc[[875, 874], "Superficie totale"].mean()

# Update the row 875 with the mean values
data.loc[875, "Superficie vendita"] = math.ceil(mean_superficie_vendita)
data.loc[875, "Superficie totale"] = math.ceil(mean_superficie_totale)

# Drop the duplicate row (index 874)
data = data.drop(index=874)

In [ ]:
data.loc[[875]]

In [ ]:
# display candidate duplicates
data.loc[[609, 608]]

In [ ]:
# Drop line 608
# We retained duplicate 609 because it aligns with the most recent 'Insegna' name displayed on the external source Google Maps (accessed on January 10, 2025)
data = data.drop([608])

In [ ]:
# display candidate duplicates
data.loc[[692, 691]]

In [ ]:
# Drop line 692.
#We retained duplicate 691 because it aligns with the most recent 'Insegna' name displayed on the external source Google Maps (accessed on January 10, 2025)
data = data.drop([692])

method 2: indexing with Sorted Neighborhood

In [ ]:
# Sort the record in the database on the basis of a key (Attribute/attributes)
data1['Key'] = data['Tipo Via'] + data['Via'] + data['Civico']

indexer = recordlinkage.index.SortedNeighbourhood(
        on='Key', window=5 #Increasing the length of the window, the number of candidate links increases as well
    )

candidate_links = indexer.index(data1)
print(len(candidate_links))
candidate_links

In [ ]:
# definition of the rules to compare the candidate pairs
compare_cl = recordlinkage.Compare()

compare_cl.exact('Via', 'Via', label='Via')
compare_cl.exact('Civico', 'Civico', label='Civico')
compare_cl.exact('ZD', 'ZD', label='ZD')
compare_cl.exact('Codice Via', 'Codice Via', label='Codice Via')
compare_cl.exact('Settore Merceologico', 'Settore Merceologico', label='Settore Merceologico')
compare_cl.string('Insegna', 'Insegna', method='jarowinkler', threshold=0.80, label='Insegna')
compare_cl.numeric('Superficie vendita', 'Superficie vendita', method='linear', offset=15, label='Superficie vendita')
compare_cl.numeric('Superficie altri usi', 'Superficie altri usi', method='linear', offset=15, label='Superficie altri usi')
compare_cl.numeric('Superficie totale', 'Superficie totale', method='linear', offset=30, label='Superficie totale')
features = compare_cl.compute(candidate_links, data1)
features

In [ ]:
# Count and sort the number of record pairs by their total similarity score
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
# compare the matches
matches = features[features.sum(axis=1) > 7]

print(len(matches))
matches
# Note: This method, when computed before the deduplication of rows, identifies the same pairs of the Blocking method, along with one additional pair.

In [ ]:
# Display the candidate duplicates
data.loc[[555, 554]] # They are not duplicates because 'Civico' values are different

# DATA QUALITY ASSESSMENT (again)

COMPLETENESS :

In [ ]:
# total number of null values
NULL = data.isnull().sum().sum()
# total number of values in the dataset
TOT = data.shape[0]*data.shape[1]

In [ ]:
# completeness evaluation
new_COMPLETENESS = '{0:.1f}%'.format((TOT - NULL) / TOT * 100)
print(new_COMPLETENESS)

ACCURACY of 'Superficie Totale' :

In [ ]:
RANGE_INCORRECT = range(0,250) # range taken from the dataset documentation
# number of out of bound values
INCORRECT_Superficie_Totale = sum(1 for item in data['Superficie totale'] if item in RANGE_INCORRECT)
# number of not null values
NOT_NULL_Superficie_totale= data['Superficie totale'].count()

In [ ]:
# accuracy evaluation
new_ACCURACY_SUPERFICIE_TOT = (data.shape[0]-INCORRECT_Superficie_Totale)/NOT_NULL_Superficie_totale
new_ACCURACY_SUPERFICIE_TOT = '{0:.1f}%'.format(new_ACCURACY_SUPERFICIE_TOT*100)
print(new_ACCURACY_SUPERFICIE_TOT)

CONSISTENCY :

In [ ]:
# integrity rule: superficie vendita + superficie altri usi = superficie totale
data['consistency'] = np.where(data['Superficie vendita'] + data['Superficie altri usi'].fillna(0) != data['Superficie totale'], 0, 1)
# number of rows that are consistent
CONSISTENT = data['consistency'].sum()
# count consistent rows
COUNT = data['consistency'].count()

In [ ]:
#cinsistency evaluation
new_CONSISTENCY = CONSISTENT / COUNT
new_CONSISTENCY = '{0:.1f}%'.format(new_CONSISTENCY * 100)
print(new_CONSISTENCY)

In [ ]:
# drop the 'consistency' column
data=data.drop(columns='consistency')

In [ ]:
# comparison of new and old DQ metrics
print("COMPLETENESS\t new: " + new_COMPLETENESS + '\t old: ' + COMPLETENESS)
print("ACCURACY\t new: " + new_ACCURACY_SUPERFICIE_TOT + '\t old: ' + ACCURACY_SUPERFICIE_TOT)
print("CONSISTENCY\t new: " + new_CONSISTENCY + '\t old: ' + CONSISTENCY)

# 7) DATA ANALYSIS

Classification on the cleaned dataset :

In [ ]:
## Remove useless columns
# for the data analysis step, information about the address (which is equivalent to an ID) is useless
data1= data.drop(columns=['Tipo Via', 'Via', 'Civico']).copy() # information about the address is still present in Codice Via and ZD.
data1

In [ ]:
#reset the index
data1 = data1.reset_index(drop=True)

In [ ]:
#function to encode categorical values
def encode(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]], dummy_na=True)
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return (res)

dataset = data1.copy()
class_name = "Settore Merceologico"

feature_cols = list(dataset.columns)
feature_cols.remove(class_name)

X = dataset[feature_cols] # Features
y = dataset[class_name] # Target variable


categorical_columns=list(X.select_dtypes(include=['bool','object']).columns)

#encode categorical coluumns
for col in X.columns:
    if col in categorical_columns:
        X = encode(X,col)

X

In [ ]:
X = StandardScaler().fit_transform(X) #scales numerical values from 0 to 1
#X = np.nan_to_num(X) is useless because we already removed all nan
X

In [ ]:

classifier = KNeighborsClassifier()

print("Training the model...")

model_fit = classifier.fit(X, y)

cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=2023)

# Perform cross-validation and evaluate using accuracy and f1
model_scores1 = cross_val_score(model_fit, X, y, cv=cv, scoring="accuracy")
model_scores2 = cross_val_score(model_fit, X, y, cv=cv, scoring="f1_macro")

accuracy_mean_clean = model_scores1.mean()
f1_mean_clean = model_scores2.mean()

print("Accuracy performance: " + str(accuracy_mean_clean))
print("F1 Score performance: " + str(f1_mean_clean))

same classification analysis with the dirty dataset :

In [ ]:
# load original dirty dataset
original_data = pd.read_csv("https://raw.githubusercontent.com/SusannaPaoletti/DataQuality/refs/heads/main/Comune-di-Milano-Attivita-commerciali-di-media-e-grande-distribuzione.csv", sep=';', encoding='utf-16')
original_data

In [ ]:
dataset = original_data.drop(columns=['Ubicazione', 'Tipo Via', 'Via', 'Civico']).copy()
class_name = "Settore Merceologico"

feature_cols = list(dataset.columns)
feature_cols.remove(class_name)

X = dataset[feature_cols] # Features
y = dataset[class_name] # Target variable


# fill missing values of Settore Merceologico with mode
y = y.fillna(y.mode()[0])


categorical_columns=list(X.select_dtypes(include=['bool','object']).columns)

#encode categorical columns
for col in X.columns:
    if col in categorical_columns:
        X = encode(X,col)

y

In [ ]:
X = StandardScaler().fit_transform(X) #scales numerical values from 0 to 1
X = np.nan_to_num(X)
X

In [ ]:
classifier = KNeighborsClassifier() #for categorical variable

print("Training...")

model_fit = classifier.fit(X, y)

cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=2023)

# Perform cross-validation and evaluate using accuracy and f1
model_scores1 = cross_val_score(model_fit, X, y, cv=cv, scoring="accuracy")
model_scores2 = cross_val_score(model_fit, X, y, cv=cv, scoring="f1_macro")

accuracy_mean_dirty = model_scores1.mean()
f1_mean_dirty = model_scores2.mean()

print("Accuracy performance: " + str(accuracy_mean_dirty))
print("F1 Score performance: " + str(f1_mean_dirty))

In [ ]:
#f1 score comparison
print("F1 metric of dirty dataset: ", f1_mean_dirty)
print("F1 metric of clean dataset: ", f1_mean_clean)

In [ ]:
# save a DataFrame as a CSV
data.to_csv('data_cleaned.csv', index=False)